## RNN DDPG  
This code is from tf-agents library with minor alterations.  
PC1 stands for parameter configuration 1, more suited to our environment.

In [1]:
# coding=utf-8
# Copyright 2020 The TF-Agents Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lint as: python2, python3
r"""Train and Eval DDPG.

To run:

```bash
tensorboard --logdir $HOME/tmp/ddpg_rnn/dm/CartPole-Balance/ --port 2223 &

python tf_agents/agents/ddpg/examples/v2/train_eval_rnn.py \
  --root_dir=$HOME/tmp/ddpg_rnn/dm/CartPole-Balance/ \
  --num_iterations=100000 \
  --alsologtostderr
```
"""

root_dir = '~/Masterarbeit/rnn'

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import time

from absl import app
from absl import logging

import gin
from six.moves import range
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import

from tf_agents.agents.ddpg import actor_rnn_network
from tf_agents.agents.ddpg import critic_rnn_network
from tf_agents.agents.ddpg import ddpg_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import suite_dm_control
from tf_agents.environments import tf_py_environment
from tf_agents.environments import wrappers
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

import numpy 
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import policy_saver
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential

from RNN_Env import FREnv
max_episode_length=1000

In [2]:
def compute_avg_return(environment, policy, num_episodes=50, verbose=False):
  total_return = 0.0
  cullsteps = 0 
  total_culls = 0
  for e in range(num_episodes):

    time_step = environment.reset()
    if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
        policy_state = policy.get_initial_state() # remember where in the script we were
    else:
        #print(policy.get_initial_state(batch_size=train_env.batch_size()))
        policy_state = policy.get_initial_state(batch_size=1) # other policies without memory
    episode_return = 0.0
    i=0
    while not time_step.is_last():
        i+=1
        action_step = policy.action(time_step, policy_state)
        if action_step.action[0][0] > 0 or action_step.action[0][1] > 0:
            cullsteps += 1
            total_culls += 1
        elif action_step.action[0][0] > 0 and action_step.action[0][1] > 0:
            total_culls += 2
        policy_state = action_step.state
        time_step = environment.step(action_step.action)
        if isinstance(environment, FREnv):
            state = environment.get_state()
        else:
            state = None # TF environment from wrapper does not have get_state()
        episode_return += time_step.reward
        if verbose:
            print (f"episode {e:>2} step{i:>4} action: ", action_step.action, 
                   "state=", state, "obs=", time_step.observation, "reward=", time_step.reward)
    total_return += episode_return

  avg_return = total_return / num_episodes
  cullsteps /= num_episodes
  total_culls /= num_episodes
  cull_percent = cullsteps / total_culls
  return avg_return, cullsteps, cull_percent

In [3]:
@gin.configurable
def train_eval(
    root_dir,
    env_name='cartpole',
    task_name='balance',
    observations_allowlist='position',
    num_iterations=10000,
    actor_fc_layers=(400, 300),
    actor_output_fc_layers=(100,),
    actor_lstm_size=(40,),
    critic_obs_fc_layers=(400,),
    critic_action_fc_layers=None,
    critic_joint_fc_layers=(300,),
    critic_output_fc_layers=(100,),
    critic_lstm_size=(40,),
    # Params for collect
    initial_collect_episodes=100,    #1
    collect_episodes_per_iteration=4,    #1
    replay_buffer_capacity=100000,
    ou_stddev=0.2,
    ou_damping=0.15,
    # Params for target update
    target_update_tau=0.05,
    target_update_period=5,
    # Params for train
    # Params for train
    train_steps_per_iteration=100,    #200
    batch_size=64,
    train_sequence_length=50,    #10
    actor_learning_rate=1e-4,
    critic_learning_rate=1e-3,
    dqda_clipping=None,
    td_errors_loss_fn=None,
    gamma=0.99,    #.995
    reward_scale_factor=1.0,
    gradient_clipping=None,
    use_tf_functions=True,
    # Params for eval
    num_eval_episodes=100,    #10
    eval_interval=2000,    #1000
    # Params for checkpoints, summaries, and logging
    log_interval=1000,
    summary_interval=1000,
    summaries_flush_secs=10,
    debug_summaries=True,
    summarize_grads_and_vars=True,
    eval_metrics_callback=None):

  """A simple train and eval for DDPG."""

  best_return = -10000
  root_dir = os.path.expanduser(root_dir)
  train_dir = os.path.join(root_dir, 'train')
  eval_dir = os.path.join(root_dir, 'eval')
  policy_dir = os.path.join(root_dir, 'policy')

  train_summary_writer = tf.compat.v2.summary.create_file_writer(
      train_dir, flush_millis=summaries_flush_secs * 1000)
  train_summary_writer.set_as_default()

  eval_summary_writer = tf.compat.v2.summary.create_file_writer(
      eval_dir, flush_millis=summaries_flush_secs * 1000)
  eval_metrics = [
      tf_metrics.AverageReturnMetric(buffer_size=num_eval_episodes),
      tf_metrics.AverageEpisodeLengthMetric(buffer_size=num_eval_episodes)
  ]

  global_step = tf.compat.v1.train.get_or_create_global_step()
  with tf.compat.v2.summary.record_if(
      lambda: tf.math.equal(global_step % summary_interval, 0)):
    if observations_allowlist is not None:
      env_wrappers = [
          functools.partial(
              wrappers.FlattenObservationsWrapper,
              observations_allowlist=[observations_allowlist])
      ]
    else:
      env_wrappers = []

    tf_env = tf_py_environment.TFPyEnvironment(
        FREnv(herd_sizes = [32,32], expected_episode_length=100, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.02, rand_infection_prob = 0.06))
    eval_tf_env = tf_py_environment.TFPyEnvironment(
        FREnv(herd_sizes = [32,32], expected_episode_length=100, max_episode_length=max_episode_length,
               rand_recovery_prob = 0.02, rand_infection_prob = 0.06))

    actor_net = actor_rnn_network.ActorRnnNetwork(
        tf_env.time_step_spec().observation,
        tf_env.action_spec(),
        input_fc_layer_params=actor_fc_layers,
        lstm_size=actor_lstm_size,
        output_fc_layer_params=actor_output_fc_layers)

    critic_net_input_specs = (tf_env.time_step_spec().observation,
                              tf_env.action_spec())

    critic_net = critic_rnn_network.CriticRnnNetwork(
        critic_net_input_specs,
        observation_fc_layer_params=critic_obs_fc_layers,
        action_fc_layer_params=critic_action_fc_layers,
        joint_fc_layer_params=critic_joint_fc_layers,
        lstm_size=critic_lstm_size,
        output_fc_layer_params=critic_output_fc_layers,
    )

    tf_agent = ddpg_agent.DdpgAgent(
        tf_env.time_step_spec(),
        tf_env.action_spec(),
        actor_network=actor_net,
        critic_network=critic_net,
        actor_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=actor_learning_rate),
        critic_optimizer=tf.compat.v1.train.AdamOptimizer(
            learning_rate=critic_learning_rate),
        ou_stddev=ou_stddev,
        ou_damping=ou_damping,
        target_update_tau=target_update_tau,
        target_update_period=target_update_period,
        dqda_clipping=dqda_clipping,
        td_errors_loss_fn=td_errors_loss_fn,
        gamma=gamma,
        reward_scale_factor=reward_scale_factor,
        gradient_clipping=gradient_clipping,
        debug_summaries=debug_summaries,
        summarize_grads_and_vars=summarize_grads_and_vars,
        train_step_counter=global_step)
    tf_agent.initialize()

    train_metrics = [
        tf_metrics.NumberOfEpisodes(),
        tf_metrics.EnvironmentSteps(),
        tf_metrics.AverageReturnMetric(),
        tf_metrics.AverageEpisodeLengthMetric(),
    ]

    eval_policy = tf_agent.policy
    collect_policy = tf_agent.collect_policy
    
    saver = policy_saver.PolicySaver(eval_policy)

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        tf_agent.collect_data_spec,
        batch_size=tf_env.batch_size,
        max_length=replay_buffer_capacity)

    initial_collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
        tf_env,
        collect_policy,
        observers=[replay_buffer.add_batch] + train_metrics,
        num_episodes=initial_collect_episodes)

    collect_driver = dynamic_episode_driver.DynamicEpisodeDriver(
        tf_env,
        collect_policy,
        observers=[replay_buffer.add_batch] + train_metrics,
        num_episodes=collect_episodes_per_iteration)

    if use_tf_functions:
      initial_collect_driver.run = common.function(initial_collect_driver.run)
      collect_driver.run = common.function(collect_driver.run)
      tf_agent.train = common.function(tf_agent.train)

    # Collect initial replay data.
    logging.info(
        'Initializing replay buffer by collecting experience for %d episodes '
        'with a random policy.', initial_collect_episodes)
    initial_collect_driver.run()

    results = metric_utils.eager_compute(
        eval_metrics,
        eval_tf_env,
        eval_policy,
        num_episodes=num_eval_episodes,
        train_step=global_step,
        summary_writer=eval_summary_writer,
        summary_prefix='Metrics',
    )
    if eval_metrics_callback is not None:
      eval_metrics_callback(results, global_step.numpy())
    metric_utils.log_metrics(eval_metrics)

    time_step = None
    policy_state = collect_policy.get_initial_state(tf_env.batch_size)

    timed_at_step = global_step.numpy()
    time_acc = 0

    # Dataset generates trajectories with shape [BxTx...]
    dataset = replay_buffer.as_dataset(
        num_parallel_calls=3,
        sample_batch_size=batch_size,
        num_steps=train_sequence_length + 1).prefetch(3)
    iterator = iter(dataset)

    def train_step():
      experience, _ = next(iterator)
      return tf_agent.train(experience)

    if use_tf_functions:
      train_step = common.function(train_step)

    for _ in range(num_iterations):
      start_time = time.time()
      time_step, policy_state = collect_driver.run(
          time_step=time_step,
          policy_state=policy_state,
      )
      for _ in range(train_steps_per_iteration):
        train_loss = train_step()
      time_acc += time.time() - start_time

      if global_step.numpy() % log_interval == 0:
        logging.info('step = %d, loss = %f', global_step.numpy(),
                     train_loss.loss)
        steps_per_sec = (global_step.numpy() - timed_at_step) / time_acc
        logging.info('%.3f steps/sec', steps_per_sec)
        tf.compat.v2.summary.scalar(
            name='global_steps_per_sec', data=steps_per_sec, step=global_step)
        timed_at_step = global_step.numpy()
        time_acc = 0

      for train_metric in train_metrics:
        train_metric.tf_summaries(
            train_step=global_step, step_metrics=train_metrics[:2])

      if global_step.numpy() % eval_interval == 0:
        results = metric_utils.eager_compute(
            eval_metrics,
            eval_tf_env,
            eval_policy,
            num_episodes=num_eval_episodes,
            train_step=global_step,
            summary_writer=eval_summary_writer,
            summary_prefix='Metrics',
        )
        if eval_metrics_callback is not None:
          eval_metrics_callback(results, global_step.numpy())
        metric_utils.log_metrics(eval_metrics)
        avg_return, cullsteps, cull_percent = compute_avg_return(eval_tf_env, eval_policy, num_episodes=100, verbose=False)
        print('step {0}: average return = {1:.1f} cullsteps = {2:.1f} cull_percent = {3:.1f}'.format(global_step.numpy(), 
                                                                                                     avg_return.numpy().item(), 
                                                                                                     cullsteps, cull_percent))
        if avg_return > best_return:
            if avg_return > -2000:
                best_return = avg_return
                print('Final best return: ', best_return)
                saver.save(os.path.join(policy_dir, str(global_step.numpy())))
                break
            else:
                best_return = avg_return
                print('New best return: ', best_return)
                saver.save(os.path.join(policy_dir, str(global_step.numpy())))

    return train_loss

In [ ]:
train_eval(root_dir)

Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `tf.data.Dataset.scan(...) instead


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


step 2000: average return = -7691.6 cullsteps = 100.2 cull_percent = 1.0
New best return:  tf.Tensor([-7691.5786], shape=(1,), dtype=float32)


...

/opt/conda/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:558: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
INFO:tensorflow:Assets written to: /home/jovyan/Masterarbeit/rnn/policy/56000/assets


step 58000: average return = -5303.5 cullsteps = 90.5 cull_percent = 1.0
step 60000: average return = -4501.4 cullsteps = 83.7 cull_percent = 1.0
step 62000: average return = -3889.4 cullsteps = 80.0 cull_percent = 1.0
step 64000: average return = -6032.1 cullsteps = 108.3 cull_percent = 1.0
step 66000: average return = -5207.3 cullsteps = 86.1 cull_percent = 1.0
step 68000: average return = -4386.6 cullsteps = 80.4 cull_percent = 1.0
step 70000: average return = -3382.2 cullsteps = 76.7 cull_percent = 1.0
step 72000: average return = -3964.0 cullsteps = 72.9 cull_percent = 1.0
step 74000: average return = -3923.2 cullsteps = 79.5 cull_percent = 1.0
step 76000: average return = -4471.5 cullsteps = 81.7 cull_percent = 1.0
step 78000: average return = -4715.0 cullsteps = 82.7 cull_percent = 1.0
step 80000: average return = -6991.3 cullsteps = 97.6 cull_percent = 1.0
step 82000: average return = -4244.8 cullsteps = 80.3 cull_percent = 1.0
step 84000: average return = -5390.9 cullsteps = 7

step 106000: average return = -2692.1 cullsteps = 46.8 cull_percent = 1.0
New best return:  tf.Tensor([-2692.087], shape=(1,), dtype=float32)
INFO:tensorflow:Assets written to: /home/jovyan/Masterarbeit/rnn/policy/106000/assets


/opt/conda/lib/python3.9/site-packages/tensorflow/python/saved_model/nested_structure_coder.py:558: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "
INFO:tensorflow:Assets written to: /home/jovyan/Masterarbeit/rnn/policy/106000/assets


step 108000: average return = -6974.0 cullsteps = 64.7 cull_percent = 1.0
step 110000: average return = -4185.1 cullsteps = 50.0 cull_percent = 1.0
step 112000: average return = -4569.9 cullsteps = 48.3 cull_percent = 1.0
step 114000: average return = -4847.2 cullsteps = 43.9 cull_percent = 1.0
step 116000: average return = -3779.0 cullsteps = 37.4 cull_percent = 1.0
step 118000: average return = -5438.2 cullsteps = 44.0 cull_percent = 1.0
step 120000: average return = -4903.9 cullsteps = 42.2 cull_percent = 1.0
step 122000: average return = -5556.3 cullsteps = 46.0 cull_percent = 1.0
step 124000: average return = -7225.3 cullsteps = 56.9 cull_percent = 1.0
step 126000: average return = -3987.9 cullsteps = 42.2 cull_percent = 1.0
step 128000: average return = -6495.0 cullsteps = 50.0 cull_percent = 1.0
step 130000: average return = -5217.1 cullsteps = 45.4 cull_percent = 1.0
step 132000: average return = -3295.8 cullsteps = 36.9 cull_percent = 1.0
step 134000: average return = -5331.9 